Limitation(s) of sklearn’s non-negative matrix factorization library. 

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF

In [3]:
users = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/users.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/movies.csv')
train = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/test.csv')
#upload csv files from github

In [4]:
users.head(3)

,uID,gender,age,accupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [5]:
movies.head(3)

,mID,title,year,Doc,Com,Hor,Adv,Wes,Dra,Ani,...,Chi,Cri,Thr,Sci,Mys,Rom,Fil,Fan,Act,Mus
0,1,Toy Story,1995,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,3,Grumpier Old Men,1995,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
train.head(3)

,uID,mID,rating
0,744,1210,5
1,3040,1584,4
2,1451,1293,5


In [7]:
test.head(3)

,uID,mID,rating
0,2233,440,4
1,4274,587,5
2,2498,454,3


Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE. You should use sklearn library. [10 pts]

Make sure that your notebook includes the following:

use's sklearn's non-negative matrix factorization

notebook shows the RMSE with an analysis of what that RMSE means

In [8]:
allusers = list(users['uID'])
allmovies = list(movies['mID'])
mid2idx = dict(zip(movies.mID,list(range(len(movies)))))
uid2idx = dict(zip(users.uID,list(range(len(users)))))
ind_movie = [mid2idx[x] for x in train.mID] 
ind_user = [uid2idx[x] for x in train.uID]
rating_train = list(train.rating)
Mr = np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(allusers), len(allmovies))).toarray())


In [9]:
Mr

array([[5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0]])

In [10]:
len(Mr.nonzero()[0]) / float(Mr.shape[0] * Mr.shape[1])

0.029852745794625237

In [11]:
model = NMF(n_components=20)     
W = model.fit_transform(Mr)
H = model.components_

/Users/home/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [12]:
ind_movie_test = [mid2idx[x] for x in test.mID] 
ind_user_test = [uid2idx[x] for x in test.uID]
rating_test = list(test.rating)
Mr_test = np.array(coo_matrix((rating_test, (ind_user_test, ind_movie_test)), shape=(len(allusers), len(allmovies))).toarray())
    

In [13]:
Mr_test


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
len(Mr_test.nonzero()[0]) / float(Mr_test.shape[0] * Mr_test.shape[1])

0.012794052185362243

In [15]:
Mr_pred = H.T.dot(W.T).T

In [16]:
Mr_pred

array([[1.78146912e+00, 5.35925885e-01, 1.65764085e-02, ...,
        1.29782937e-02, 6.39692718e-03, 9.20804028e-02],
       [1.27770887e+00, 3.71662714e-01, 1.33311388e-01, ...,
        1.58592320e-02, 0.00000000e+00, 3.80284936e-02],
       [6.81899415e-01, 1.45064744e-01, 2.83368754e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.65793176e-01, 1.33256754e-02, 7.65224322e-04, ...,
        1.26670276e-03, 0.00000000e+00, 1.65118220e-04],
       [1.26090229e+00, 2.87961415e-01, 9.41655665e-02, ...,
        4.92756103e-02, 0.00000000e+00, 0.00000000e+00],
       [1.36679791e+00, 9.79111344e-02, 5.80462730e-03, ...,
        9.30176001e-02, 9.10052209e-02, 4.16067928e-01]])

In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
rmse = np.sqrt(mean_squared_error(Mr_pred[Mr_test.nonzero()].flatten(), Mr_test[Mr_test.nonzero()].flatten()))
  

In [19]:
rmse

2.862398742855173

Discuss the results and why they did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it?

- My RMSE result came out to 2.86. Based on last week's assignment on RMSE, a baseline should be around 1.26.
- NMF did not take TRUE movie and user features into account
- I believe tuning the number of components could improve the performance